In [1]:
import numpy as np
import pandas as pd
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine

In [45]:
bills=pd.read_csv('Data/bills.csv')
committee_members=pd.read_csv('Data/committee_members.csv')
committees=pd.read_csv('Data/committees.csv')
congress_members=pd.read_csv('Data/congress_members.csv')
leadership=pd.read_csv('Data/leadership.csv')
os_contributions=pd.read_csv('Data/os_contributions.csv')
os_legislators=pd.read_csv('Data/os_legislators.csv')
terms=pd.read_csv('Data/terms.csv')
votingaffinity=pd.read_csv('Data/votingaffinity.csv')

# Bills

In [46]:
bills.columns = [x.lower() for x in bills.columns]
bills.columns = [x.replace('.', '_') for x in bills.columns]
bills.columns

Index(['congress', 'introduceddate', 'number', 'title', 'type', 'url',
       'latestaction_actiondate', 'latestaction_text', 'policyarea_name',
       'amendmentnumber', 'latestaction', 'latestaction_actiontime'],
      dtype='object')

# Congress Members

In [47]:
congress_members.columns = [x.lower() for x in congress_members.columns]
congress_members.columns = [x.replace('.', '_') for x in congress_members.columns]
congress_members.columns

Index(['bioguideid', 'birthyear', 'currentmember', 'directordername',
       'firstname', 'honorificname', 'invertedordername', 'lastname',
       'officialwebsiteurl', 'partyhistory', 'state', 'terms', 'updatedate',
       'addressinformation_city', 'addressinformation_district',
       'addressinformation_officeaddress', 'addressinformation_phonenumber',
       'addressinformation_zipcode', 'cosponsoredlegislation_count',
       'cosponsoredlegislation_url', 'depiction_attribution',
       'depiction_imageurl', 'sponsoredlegislation_count',
       'sponsoredlegislation_url', 'middlename', 'suffixname', 'nickname',
       'leadership', 'district'],
      dtype='object')

In [48]:
congress_members = congress_members.drop(['terms', 'leadership', 'partyhistory'], axis=1)

In [49]:
# Terms

In [50]:
terms.columns = [x.lower() for x in terms.columns]
terms.columns = [x.replace('.', '_') for x in terms.columns]
terms.columns

Index(['chamber', 'congress', 'endyear', 'membertype', 'startyear',
       'statecode', 'statename', 'bioguideid', 'district'],
      dtype='object')

In [51]:
# Leadership

In [52]:
leadership.columns = [x.lower() for x in leadership.columns]
leadership.columns = [x.replace('.', '_') for x in leadership.columns]
leadership.head(3).T

,0,1,2
congress,113,114,115
type,Assistant Democratic Leader,Assistant Majority Leader,Majority Whip
bioguideid,C001056,C001056,C001056
current,NaN,NaN,NaN


# Os_legislators

In [53]:
os_legislators.columns = [x.lower() for x in os_legislators.columns]
os_legislators.columns = [x.replace('.', '_') for x in os_legislators.columns]
os_legislators.head(3).T

,0,1,2
cid,N00050780,N00035774,N00026050
firstl,Mary Peltola,Dan Sullivan,Lisa Murkowski
lastnam,Peltola,Sullivan,Murkowski
party,D,R,R
offic,AK01,AKS1,AKS2
gend,F,M,F
first_elected,2022,2014,2002
xit_cod,0,0,0
commen,NaN,NaN,NaN
phon,NaN,202-224-3004,202-224-6665


In [54]:
os_legislators = os_legislators[['cid', 'party','oguide_id']]
os_legislators = os_legislators.rename({'oguide_id': 'bioguideid'}, axis=1)

In [55]:
members = pd.merge(congress_members, os_legislators, on = 'bioguideid',
                  how = 'outer', validate = 'one_to_many', indicator= 'matched')

In [56]:
members['matched'].value_counts()

matched
both          506
left_only      31
right_only     31
Name: count, dtype: int64

In [57]:
members = pd.merge(congress_members, os_legislators, on = 'bioguideid',
                  how = 'inner')

In [58]:
os_contributions.columns = [x.lower() for x in os_contributions.columns]
os_contributions.columns = [x.replace('.', '_') for x in os_contributions.columns]
os_contributions.head(30)

,org_nam,otal,pac,ndiv
0,State of Alaska,19541,0,19541
1,"Sonosky, Chambers et al",16050,0,16050
2,General Communication Inc,15150,2500,12650
3,Google Inc,14164,0,14164
4,University of Alaska/Anchorage,13938,0,13938
5,Swing Left,13400,0,13400
6,Apple Inc,10783,0,10783
7,Service Employees International Union,10010,5000,5010
8,National Education Assn,10000,10000,0
9,Sealaska Corp,9950,0,9950


# Committees

In [59]:
committees.columns = [x.lower() for x in committees.columns]
committees.columns = [x.replace('.', '_') for x in committees.columns]
committees = committees.drop(['subcommittees'], axis = 1)
committees.head(30)

,chamber,committeetypecode,name,systemcode,url,parent_name,parent_systemcode,parent_url
0,House,Other,Bicentenary Committee,hcza00,https://api.congress.gov/v3/committee/house/hc...,NaN,NaN,NaN
1,House,Standing,Energy (Ad Hoc) Committee,hhah00,https://api.congress.gov/v3/committee/house/hh...,NaN,NaN,NaN
2,House,Select,U.S. Role in Iranian Arms Committee,hlbz00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN
3,House,Select,"Children, Youth, and Families (Select) Committee",hlcf00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN
4,House,Select,Select Committee on the Climate Crisis,hlcn00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN
5,House,Select,Select Committee on Committees Committee,hlcq00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN
6,House,Select,Select Committee on Ethics (105th),hleb00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN
7,House,Select,Select Committee on Economic Disparity and Fai...,hlef00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN
8,House,Select,Ethics (Select) Committee,hlet00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN
9,House,Select,Select Subcommittee on the Weaponization of th...,hlfd00,https://api.congress.gov/v3/committee/house/hl...,Judiciary Committee,hsju00,https://api.congress.gov/v3/committee/house/hs...


# Committee Members

In [60]:
committee_members.columns = [x.lower() for x in committee_members.columns]
committee_members.columns = [x.replace('.', '_') for x in committee_members.columns]
committee_members.head(30)

,committee_code,rank,title,bioguide
0,HSII,1.0,Chair,W000821
1,HSII,1.0,Ranking Member,G000551
2,HSII,2.0,NaN,L000564
3,HSII,2.0,NaN,N000179
4,HSII,3.0,NaN,W000804
5,HSII,3.0,NaN,S001177
6,HSII,4.0,NaN,M001177
7,HSII,4.0,NaN,H001068
8,HSII,5.0,NaN,G000565
9,HSII,5.0,NaN,G000574
